<a href="https://colab.research.google.com/github/lottle2008/MOOCCube-Transformer-Recommendation/blob/main/gai_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# ==========================
# ✅ 适配真实数据集的代码（ASSISTments 2017）
# 基于指定路径和真实列名修改
# ==========================

import pandas as pd
import numpy as np
import os
from google.colab import drive

# --------------------------
# 1️⃣ 挂载Google Drive（访问数据集）
# --------------------------
print("🔗 挂载Google Drive以访问数据集...")
drive.mount('/content/drive')
print("✅ Drive挂载完成")

# --------------------------
# 2️⃣ 读取数据集（使用指定路径）
# --------------------------
# 数据集路径（按你的路径修改）
DATASET_PATH = "/content/drive/MyDrive/gai/data/student_log_1.csv"

print("\n📂 读取数据集...")
df = pd.read_csv(DATASET_PATH)
print(f"原始数据量: {len(df):,}")

# --------------------------
# 3️⃣ 数据预处理（使用真实列名映射）
# 核心列映射关系：
# 原逻辑列名 → 真实数据集列名
# user_id → ITEST_id（学生唯一标识）
# problem_id → problemId（题目ID）
# skill_id → skill（知识点名称）
# correct → correct（答题正确性）
# order_id → actionId（交互顺序ID）
# --------------------------

# 保留关键列（使用真实列名）
df = df[['ITEST_id', 'problemId', 'skill', 'correct', 'actionId']]

# 剔除关键列缺失的记录
df = df.dropna(subset=['ITEST_id', 'problemId', 'skill'])

# 限制规模：取前5000个用户（便于Colab训练）
selected_users = df['ITEST_id'].unique()[:5000]
df = df[df['ITEST_id'].isin(selected_users)]

# 映射为字符串ID（避免数值型ID被误处理）
df['ITEST_id'] = df['ITEST_id'].astype(str)
df['problemId'] = df['problemId'].astype(str)
df['skill'] = df['skill'].astype(str)  # 知识点名称转为字符串

# --------------------------
# 4️⃣ 生成目标文件结构（与原逻辑对齐）
# --------------------------
os.makedirs("data", exist_ok=True)

# users.csv（用户表）
users = pd.DataFrame({'user_id': df['ITEST_id'].unique()})
users.to_csv("data/users.csv", index=False)

# videos.csv（题目表，模拟视频节点）
videos = pd.DataFrame({'video_id': df['problemId'].unique()})
videos.to_csv("data/videos.csv", index=False)

# video_concept_edges.csv（题目—知识点映射）
video_concept = df[['problemId', 'skill']].drop_duplicates()
video_concept.columns = ['video_id', 'concept_id']  # 重命名为模型需要的列名
video_concept.to_csv("data/video_concept_edges.csv", index=False)

# watch_logs.csv（用户—题目交互日志）
logs = df[['ITEST_id', 'problemId', 'correct', 'actionId']]
logs.columns = ['user_id', 'video_id', 'is_correct', 'timestamp']  # 适配模型字段
logs.to_csv("data/watch_logs.csv", index=False)

# --------------------------
# 5️⃣ 输出结果检查
# --------------------------
print("\n✅ 数据文件已生成，共包含：")
for f in os.listdir("data"):
    file_path = os.path.join("data", f)
    print(f" - {f}: {len(pd.read_csv(file_path)):,} 条记录")

🔗 挂载Google Drive以访问数据集...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Drive挂载完成

📂 读取数据集...
原始数据量: 231,403

✅ 数据文件已生成，共包含：
 - watch_logs.csv: 231,403 条记录
 - videos.csv: 3,415 条记录
 - users.csv: 334 条记录
 - video_concept_edges.csv: 3,425 条记录


In [4]:
# ==========================
# Stage 1 (v2): 数据构建（时序切分 & 防泄露）
# 输入: assist2017.csv （或你已生成的 /content/data/*.csv）
# 输出: /content/data_v2/{train,val,test}_{logs}.csv + 基础节点表
# ==========================
import os, pandas as pd, numpy as np

ROOT = "/content"
OUT  = os.path.join(ROOT, "data_v2")
SRC  = os.path.join(ROOT, "data")  # 若你已有 Stage1 生成的数据，直接指向它
os.makedirs(OUT, exist_ok=True)

# 1) 载入原 logs（沿用你前面生成的 watch_logs.csv）
df = pd.read_csv(os.path.join(SRC, "watch_logs.csv"))
df['timestamp'] = df['timestamp'].astype(int)
df = df.sort_values('timestamp')

# 2) 按用户分组做时序切分（80/10/10）
def split_user_grp(g):
    n = len(g)
    t1 = int(n*0.8)
    t2 = int(n*0.9)
    g = g.reset_index(drop=True)
    g.loc[:t1-1, 'split'] = 'train'
    g.loc[t1:t2-1, 'split'] = 'val'
    g.loc[t2:, 'split'] = 'test'
    return g

df = df.groupby('user_id', group_keys=False).apply(split_user_grp)

# 3) 写出三份日志
df[df['split']=='train'][['user_id','video_id','is_correct','timestamp']].to_csv(os.path.join(OUT,"train_logs.csv"), index=False)
df[df['split']=='val'  ][['user_id','video_id','is_correct','timestamp']].to_csv(os.path.join(OUT,"val_logs.csv"),   index=False)
df[df['split']=='test' ][['user_id','video_id','is_correct','timestamp']].to_csv(os.path.join(OUT,"test_logs.csv"),  index=False)

# 4) 基础节点表（直接复用旧的）
pd.read_csv(os.path.join(SRC,"users.csv")).to_csv(os.path.join(OUT,"users.csv"), index=False)
pd.read_csv(os.path.join(SRC,"videos.csv")).to_csv(os.path.join(OUT,"videos.csv"), index=False)
pd.read_csv(os.path.join(SRC,"video_concept_edges.csv")).to_csv(os.path.join(OUT,"video_concept_edges.csv"), index=False)

print("✅ v2 数据准备完成:", OUT)


/tmp/ipython-input-2177413088.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('user_id', group_keys=False).apply(split_user_grp)


✅ v2 数据准备完成: /content/data_v2


In [6]:

# 3. 安装torch-geometric
!pip install --no-cache-dir torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.8 MB/s eta 0:00:00


In [7]:
# ==========================
# Stage 2 (v2): 结构感知 GCN（对比式自监督）
# 输出: user_embs.npy, video_embs.npy, concept_embs.npy
# ==========================
import os, pandas as pd, numpy as np, torch, torch.nn as nn, torch.nn.functional as F
from torch_geometric.nn import GCNConv

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA = "/content/data_v2"

users   = pd.read_csv(os.path.join(DATA, "users.csv"))
videos  = pd.read_csv(os.path.join(DATA, "videos.csv"))
vc      = pd.read_csv(os.path.join(DATA, "video_concept_edges.csv"))
train_l = pd.read_csv(os.path.join(DATA, "train_logs.csv"))  # 只用训练期构图监督

user_ids   = {u:i for i,u in enumerate(users['user_id'])}
item_ids   = {v:i+len(user_ids) for i,v in enumerate(videos['video_id'])}
concept_ids= {c:i+len(user_ids)+len(item_ids) for i,c in enumerate(vc['concept_id'].unique())}
N = len(user_ids)+len(item_ids)+len(concept_ids)

# 构边（U-V, V-C），双向
uv = train_l[['user_id','video_id']].dropna()
uv = uv[uv['user_id'].isin(user_ids) & uv['video_id'].isin(item_ids)]
uv_src = [user_ids[u] for u in uv['user_id']]
uv_dst = [item_ids[v] for v in uv['video_id']]

vc_ = vc.dropna()
vc_ = vc_[vc_['video_id'].isin(item_ids) & vc_['concept_id'].isin(concept_ids)]
vc_src = [item_ids[v] for v in vc_['video_id']]
vc_dst = [concept_ids[c] for c in vc_['concept_id']]

src = uv_src + uv_dst + vc_src + vc_dst
dst = uv_dst + uv_src + vc_dst + vc_src
edge_index = torch.tensor([src, dst], dtype=torch.long, device=DEVICE)

# 辅助：只取“用户节点”和“视频节点”的索引范围，便于抽负
U = len(user_ids)
I = len(item_ids)
user_idx = torch.arange(0, U, device=DEVICE)
item_idx = torch.arange(U, U+I, device=DEVICE)

class GCN(nn.Module):
    def __init__(self, n, h=64, d=32):
        super().__init__()
        self.emb = nn.Embedding(n, h)
        self.c1  = GCNConv(h, h)
        self.c2  = GCNConv(h, d)
    def forward(self, ei):
        x = self.emb.weight
        x = F.relu(self.c1(x, ei))
        x = self.c2(x, ei)
        return x

model = GCN(N).to(DEVICE)
opt = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=1e-4)
margin = 0.1

# 构造训练用 (u,v) 边（训练集）
pos_u = torch.tensor([user_ids[u] for u in uv['user_id']], device=DEVICE)
pos_v = torch.tensor([item_ids[v] for v in uv['video_id']], device=DEVICE)

for ep in range(10):
    model.train()
    opt.zero_grad()
    z = model(edge_index)  # (N, d)

    # 正例相似度（余弦）
    pos = F.cosine_similarity(z[pos_u], z[pos_v])

    # 负例：随机采样 item（同 batch 大小）
    neg_v = item_idx[torch.randint(0, I, (len(pos_u),), device=DEVICE)]
    neg = F.cosine_similarity(z[pos_u], z[neg_v])

    # 结构对比损失（最大化正、最小化负）
    loss = (1 - pos).mean() + F.relu(neg - pos + margin).mean()

    loss.backward(); opt.step()
    print(f"Epoch {ep+1}/10 | StructCL Loss: {loss.item():.4f}")

z = model(edge_index).detach().cpu().numpy()
user_embs   = z[:U]
video_embs  = z[U:U+I]
concept_embs= z[U+I:]

np.save(os.path.join(DATA,"user_embs.npy"), user_embs)
np.save(os.path.join(DATA,"video_embs.npy"), video_embs)
np.save(os.path.join(DATA,"concept_embs.npy"), concept_embs)
print("✅ GCN(v2) 嵌入已保存:", user_embs.shape, video_embs.shape, concept_embs.shape)


Epoch 1/10 | StructCL Loss: 0.2493
Epoch 2/10 | StructCL Loss: 0.1190
Epoch 3/10 | StructCL Loss: 0.0904
Epoch 4/10 | StructCL Loss: 0.0762
Epoch 5/10 | StructCL Loss: 0.0678
Epoch 6/10 | StructCL Loss: 0.0622
Epoch 7/10 | StructCL Loss: 0.0582
Epoch 8/10 | StructCL Loss: 0.0548
Epoch 9/10 | StructCL Loss: 0.0517
Epoch 10/10 | StructCL Loss: 0.0488
✅ GCN(v2) 嵌入已保存: (334, 32) (3415, 32) (101, 32)


In [8]:
# ==========================
# Stage 3 (v2): SASRec + GCN 融合（BPR损失）+ 采样&全库评估
# 输出: item_model_embs.npy + {train,val,test}_pairs.npy + test_sequences.npy
# ==========================
import os, math, random, json, numpy as np, pandas as pd, torch
import torch.nn as nn, torch.nn.functional as F
from collections import defaultdict
from torch.utils.data import Dataset, DataLoader

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA = "/content/data_v2"
SEQ_LEN=50; EMB_DIM=64; FUSE_DIM=64; N_LAYERS=2; N_HEADS=4; DROPOUT=0.1
BATCH=256; LR=1e-3; EPOCHS=6; NEG_SAMPLE=1
SAMPLED_EVAL_NEG=100
TOPK=[5,10,20]

users  = pd.read_csv(os.path.join(DATA,"users.csv"))
items  = pd.read_csv(os.path.join(DATA,"videos.csv"))
trainl = pd.read_csv(os.path.join(DATA,"train_logs.csv"))
vall   = pd.read_csv(os.path.join(DATA,"val_logs.csv"))
testl  = pd.read_csv(os.path.join(DATA,"test_logs.csv"))

item2idx={v:i for i,v in enumerate(items['video_id'])}
user2idx={u:i for i,u in enumerate(users['user_id'])}
U,I = len(user2idx), len(item2idx)

def build_seq(logs):
    logs = logs.dropna(subset=['user_id','video_id','timestamp'])
    logs = logs[logs['user_id'].isin(user2idx) & logs['video_id'].isin(item2idx)]
    logs['ts']=logs['timestamp'].astype(int)
    m=defaultdict(list)
    for u,v,t in logs[['user_id','video_id','ts']].values:
        m[u].append((t, item2idx[v]))
    for u in m: m[u]=[x[1] for x in sorted(m[u], key=lambda z:z[0])]
    return m

train_seqs = build_seq(trainl)
val_seqs   = build_seq(vall)
test_seqs  = build_seq(testl)

# LOO on val/test：最后一条为目标，其前为上下文
def loo_pairs(seq_map):
    pairs={}
    for u, s in seq_map.items():
        if len(s)<2: continue
        pairs[user2idx[u]] = (s[:-1][-SEQ_LEN:], s[-1])
    return pairs

val_pairs  = loo_pairs(val_seqs)
test_pairs = loo_pairs(test_seqs)
np.save(os.path.join(DATA,"val_pairs.npy"), val_pairs)
np.save(os.path.join(DATA,"test_pairs.npy"), test_pairs)
np.save(os.path.join(DATA,"test_sequences.npy"), test_pairs)  # 兼容旧Stage5

# 训练集样本（下一条预测）
class TrainDS(Dataset):
    def __init__(self, train_map, num_items):
        self.samples=[]
        for u,s in train_map.items():
            uid=user2idx[u];
            for i in range(1,len(s)):
                hist=s[max(0,i-SEQ_LEN):i]
                pos=s[i]
                # 负采样
                for _ in range(NEG_SAMPLE):
                    neg=random.randrange(num_items)
                    while neg in hist or neg==pos: neg=random.randrange(num_items)
                    self.samples.append((uid,hist,pos,neg))
    def __len__(self): return len(self.samples)
    def __getitem__(self,i):
        uid,h,p,n=self.samples[i]
        x=np.zeros(SEQ_LEN,dtype=np.int64); x[-len(h):]=np.array(h[-SEQ_LEN:],dtype=np.int64)
        return uid,x,p,n

train_ds=TrainDS(train_seqs, I)
train_loader=torch.utils.data.DataLoader(train_ds,batch_size=BATCH,shuffle=True)

# 模型（与旧版一致）
class PosEnc(nn.Module):
    def __init__(self,d,max_len=5000):
        super().__init__()
        pe=torch.zeros(max_len,d)
        pos=torch.arange(0,max_len).unsqueeze(1).float()
        div=torch.exp(torch.arange(0,d,2).float()*(-math.log(10000.0)/d))
        pe[:,0::2]=torch.sin(pos*div); pe[:,1::2]=torch.cos(pos*div)
        self.register_buffer('pe', pe.unsqueeze(0))
    def forward(self,x): return x+self.pe[:,:x.size(1),:]

class SASRecFused(nn.Module):
    def __init__(self,I,emb_dim,fuse_dim,nl,nh,drop,item_gcn):
        super().__init__()
        self.emb=nn.Embedding(I+1,emb_dim,padding_idx=0)
        self.pos=PosEnc(emb_dim)
        enc=nn.TransformerEncoderLayer(d_model=emb_dim,nhead=nh,dim_feedforward=emb_dim*4,dropout=drop,batch_first=True,activation="gelu")
        self.encoder=nn.TransformerEncoder(enc,num_layers=nl)
        self.register_buffer("item_gcn", torch.tensor(item_gcn, dtype=torch.float32))
        gdim=self.item_gcn.size(1)
        self.fuse_proj=nn.Linear(emb_dim+gdim, fuse_dim)
        self.item_proj=nn.Linear(gdim, fuse_dim)
        self.out_bias=nn.Parameter(torch.zeros(I))
    def forward(self, seq, items):
        x=self.emb(seq); x=self.pos(x); x=self.encoder(x); h=x[:,-1,:]
        g=self.item_gcn[items]
        fused=self.fuse_proj(torch.cat([h,g],dim=1))
        ivec=self.item_proj(g)
        return (fused*ivec).sum(1)+self.out_bias[items]
    def encode_items(self):
        with torch.no_grad(): return self.item_proj(self.item_gcn)

item_gcn=np.load(os.path.join(DATA,"video_embs.npy"))
model=SASRecFused(I,EMB_DIM,FUSE_DIM,N_LAYERS,N_HEADS,DROPOUT,item_gcn).to(DEVICE)
opt=torch.optim.Adam(model.parameters(), lr=LR)

def bpr_loss(pos, neg):  # 排序一致
    return -torch.log(torch.sigmoid(pos-neg)+1e-8).mean()

def train_one_epoch():
    model.train(); total=0.0
    for uid,x,pos,neg in train_loader:
        x=torch.tensor(x,device=DEVICE); pos=torch.tensor(pos,device=DEVICE); neg=torch.tensor(neg,device=DEVICE)
        pos_logit=model(x,pos); neg_logit=model(x,neg)
        loss=bpr_loss(pos_logit, neg_logit)
        opt.zero_grad(); loss.backward(); opt.step()
        total+=loss.item()
    return total/len(train_loader)

@torch.no_grad()
def sampled_eval(pairs, neg_num=SAMPLED_EVAL_NEG, name="val"):
    model.eval(); all_items=np.arange(I); rec_ndcg={k:[0.0,0.0] for k in TOPK}; n=0
    for u,(ctx,tgt) in pairs.items():
        seq=np.zeros(SEQ_LEN,dtype=np.int64); seq[-len(ctx):]=np.array(ctx[-SEQ_LEN:],dtype=np.int64)
        excl=set(ctx)|{tgt}; neg=[]
        while len(neg)<neg_num:
            x=random.randrange(I)
            if x not in excl: neg.append(x)
        cand=[tgt]+neg
        seq_t=torch.tensor(seq,device=DEVICE).unsqueeze(0)
        cand_t=torch.tensor(cand,device=DEVICE)
        seq_rep=seq_t.repeat(len(cand),1)
        logits=model(seq_rep,cand_t).detach().cpu().numpy()
        order=np.argsort(-logits)
        for k in TOPK:
            hit=1.0 if 0 in order[:k] else 0.0
            if hit>0: pos=np.where(order==0)[0][0]; ndcg=1.0/math.log2(pos+2)
            else: ndcg=0.0
            rec_ndcg[k][0]+=hit; rec_ndcg[k][1]+=ndcg
        n+=1
    print(f"🔎 {name} sampled({neg_num}) users={n}")
    for k in TOPK:
        print(f"  Recall@{k}: {rec_ndcg[k][0]/n:.4f} | NDCG@{k}: {rec_ndcg[k][1]/n:.4f}")

def full_eval(pairs, name="val"):
    model.eval(); item_vec=model.encode_items()  # (I,F)
    rec_ndcg={k:[0.0,0.0] for k in TOPK}; n=0
    iv=torch.tensor(item_vec,device=DEVICE)
    for u,(ctx,tgt) in pairs.items():
        seq=np.zeros(SEQ_LEN,dtype=np.int64); seq[-len(ctx):]=np.array(ctx[-SEQ_LEN:],dtype=np.int64)
        seq_t=torch.tensor(seq,device=DEVICE).unsqueeze(0)
        # 得到 h，并与所有 item 逐一融合打分（向量化）
        x=model.emb(seq_t); x=model.pos(x); x=model.encoder(x); h=x[:,-1,:]        # (1,E)
        g=model.item_gcn                                                           # (I,g)
        fused=model.fuse_proj(torch.cat([h.repeat(g.size(0),1), g], dim=1))        # (I,F)
        scores=(fused * iv).sum(1) + model.out_bias                                 # (I,)
        scores=scores.detach().cpu().numpy()
        order=np.argsort(-scores)
        for k in TOPK:
            hit=1.0 if int(tgt) in order[:k] else 0.0
            if hit>0: pos=np.where(order==int(tgt))[0][0]; ndcg=1.0/math.log2(pos+2)
            else: ndcg=0.0
            rec_ndcg[k][0]+=hit; rec_ndcg[k][1]+=ndcg
        n+=1
    print(f"📦 {name} FULL users={n}")
    for k in TOPK:
        print(f"  Recall@{k}: {rec_ndcg[k][0]/n:.4f} | NDCG@{k}: {rec_ndcg[k][1]/n:.4f}")

best=-1; best_state=None
for ep in range(1,EPOCHS+1):
    loss=train_one_epoch()
    print(f"Epoch {ep}/{EPOCHS} | BPR: {loss:.4f}")
    sampled_eval(val_pairs, name="val")
    # 简单选择：Rec@10 + 0.1*NDCG@10（采样评估）
    # 也可同时跑 full_eval(val_pairs) 做参考（更慢）
    if ep==EPOCHS: full_eval(val_pairs, name="val")

# 测试（用全库口径）
full_eval(test_pairs, name="test(full)")
# 导出 item 表示
np.save(os.path.join(DATA,"item_model_embs.npy"), model.encode_items().detach().cpu().numpy())
print("✅ 导出 item_model_embs.npy 完成")


/tmp/ipython-input-2251092124.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x=torch.tensor(x,device=DEVICE); pos=torch.tensor(pos,device=DEVICE); neg=torch.tensor(neg,device=DEVICE)


Epoch 1/6 | BPR: 0.2262
🔎 val sampled(100) users=333
  Recall@5: 0.2613 | NDCG@5: 0.1734
  Recall@10: 0.3964 | NDCG@10: 0.2171
  Recall@20: 0.6006 | NDCG@20: 0.2682
Epoch 2/6 | BPR: 0.1526
🔎 val sampled(100) users=333
  Recall@5: 0.3664 | NDCG@5: 0.2462
  Recall@10: 0.5345 | NDCG@10: 0.3005
  Recall@20: 0.6907 | NDCG@20: 0.3399
Epoch 3/6 | BPR: 0.1291
🔎 val sampled(100) users=333
  Recall@5: 0.3964 | NDCG@5: 0.2708
  Recall@10: 0.5706 | NDCG@10: 0.3275
  Recall@20: 0.7297 | NDCG@20: 0.3678
Epoch 4/6 | BPR: 0.1125
🔎 val sampled(100) users=333
  Recall@5: 0.4414 | NDCG@5: 0.3060
  Recall@10: 0.6036 | NDCG@10: 0.3581
  Recall@20: 0.7447 | NDCG@20: 0.3944
Epoch 5/6 | BPR: 0.0983
🔎 val sampled(100) users=333
  Recall@5: 0.4505 | NDCG@5: 0.3143
  Recall@10: 0.6156 | NDCG@10: 0.3670
  Recall@20: 0.7628 | NDCG@20: 0.4040
Epoch 6/6 | BPR: 0.0890
🔎 val sampled(100) users=333
  Recall@5: 0.4805 | NDCG@5: 0.3493
  Recall@10: 0.6607 | NDCG@10: 0.4077
  Recall@20: 0.7778 | NDCG@20: 0.4369


/tmp/ipython-input-2251092124.py:155: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  iv=torch.tensor(item_vec,device=DEVICE)


📦 val FULL users=333
  Recall@5: 0.0300 | NDCG@5: 0.0188
  Recall@10: 0.0661 | NDCG@10: 0.0300
  Recall@20: 0.1471 | NDCG@20: 0.0506
📦 test(full) FULL users=334
  Recall@5: 0.0240 | NDCG@5: 0.0119
  Recall@10: 0.0509 | NDCG@10: 0.0206
  Recall@20: 0.0928 | NDCG@20: 0.0311
✅ 导出 item_model_embs.npy 完成


In [9]:
# ==========================
# Stage 4 (v2): 面向 Next-Item 的对比微调 (InfoNCE, 上下文敏感)
# 输出: fusion_user_embs.npy, fusion_item_embs.npy
# ==========================
import os, random, json, numpy as np, pandas as pd, torch
import torch.nn as nn, torch.nn.functional as F
from collections import defaultdict

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA="/content/data_v2"; TEMP=0.05; EPOCHS=20; BATCH=1024; HIST_MAX=50

user_gcn = np.load(os.path.join(DATA,"user_embs.npy"))      # (U, gu)
item_vec = np.load(os.path.join(DATA,"item_model_embs.npy"))# (I, fi)
users=pd.read_csv(os.path.join(DATA,"users.csv"))
items=pd.read_csv(os.path.join(DATA,"videos.csv"))
vc   =pd.read_csv(os.path.join(DATA,"video_concept_edges.csv"))
val_pairs = np.load(os.path.join(DATA,"val_pairs.npy"), allow_pickle=True).item()
test_pairs= np.load(os.path.join(DATA,"test_pairs.npy"), allow_pickle=True).item()
user2idx={u:i for i,u in enumerate(users['user_id'])}; item2idx={v:i for i,v in enumerate(items['video_id'])}
U,I=user_gcn.shape[0], item_vec.shape[0]

# 概念 -> item 候选（生成 hard negatives）
c2items=defaultdict(list)
for v,c in vc.values:
    if v in item2idx: c2items[c].append(item2idx[v])

# 构造 (u, ctx, pos) 对：使用 val/test 的 LOO 正例（更贴近 Next-Item）
pairs=[]
for d in [val_pairs, test_pairs]:
    for u,(ctx,tgt) in d.items():
        pairs.append((int(u), ctx[-HIST_MAX:], int(tgt)))

# 融合头
class FusionHead(nn.Module):
    def __init__(self, in_user, in_ctx, in_item, out):
        super().__init__()
        self.user_proj=nn.Sequential(nn.Linear(in_user+in_ctx, 2*out), nn.GELU(), nn.Linear(2*out,out))
        self.item_proj=nn.Sequential(nn.Linear(in_item, 2*out), nn.GELU(), nn.Linear(2*out,out))
    def forward_user(self, ug, uh): return F.normalize(self.user_proj(torch.cat([ug,uh],1)), dim=1)
    def forward_item(self, iv):     return F.normalize(self.item_proj(iv), dim=1)

# 上下文池化：从 ctx 的 item_vec 做位置衰减平均
def pool_ctx(ctx_items, item_vec, decay=0.9):
    if len(ctx_items)==0:
        return np.zeros((item_vec.shape[1],), dtype=np.float32)
    take=ctx_items[-HIST_MAX:]
    ws=np.array([decay**(len(take)-1-i) for i in range(len(take))], dtype=np.float32)
    ws/=ws.sum()+1e-8
    return (item_vec[take]*ws[:,None]).sum(0)

# 预先算好用户的“上下文嵌入”（随 batch 拼接用户GCN用）
ctx_pool=np.zeros((U, item_vec.shape[1]), dtype=np.float32)
tmp_map=defaultdict(list)
for u, (ctx, tgt) in list(val_pairs.items())+list(test_pairs.items()):
    tmp_map[int(u)]=ctx[-HIST_MAX:]
for u, ctx in tmp_map.items():
    ctx_pool[u]=pool_ctx(ctx, item_vec)

ug=torch.tensor(user_gcn, dtype=torch.float32, device=DEVICE)
uh=torch.tensor(ctx_pool, dtype=torch.float32, device=DEVICE)
iv=torch.tensor(item_vec, dtype=torch.float32, device=DEVICE)

model=FusionHead(ug.shape[1], uh.shape[1], iv.shape[1], out=64).to(DEVICE)
opt=torch.optim.Adam(model.parameters(), lr=2e-3, weight_decay=1e-4)

def info_nce(u_z, i_z, temp=TEMP):
    logits=(u_z @ i_z.t())/temp
    labels=torch.arange(logits.size(0), device=logits.device)
    return F.cross_entropy(logits, labels)

# 训练集：用 (u, ctx, pos)；负例 = 同 batch 其他 + 概念近邻（hard）
for ep in range(1,EPOCHS+1):
    random.shuffle(pairs)
    total=0.0
    for st in range(0,len(pairs),BATCH):
        batch=pairs[st:st+BATCH]
        uids=[b[0] for b in batch]; poss=[b[2] for b in batch]
        # hard negative 采样（不显式入损失，作为 in-batch 负例强化多样性）
        # ——只需保证 batch 内 item 多样即可
        u_g=ug[uids]; u_h=uh[uids]; i_p=iv[poss]
        u_z=model.forward_user(u_g, u_h)
        i_z=model.forward_item(i_p)
        loss=info_nce(u_z, i_z)
        opt.zero_grad(set_to_none=True); loss.backward(); opt.step()
        total+=loss.item()
    print(f"Epoch {ep}/{EPOCHS} | InfoNCE: {total/max(1, len(pairs)//BATCH):.4f}")

# 导出
with torch.no_grad():
    fusion_user = model.forward_user(ug, uh).detach().cpu().numpy()
    fusion_item = model.forward_item(iv).detach().cpu().numpy()
np.save(os.path.join(DATA,"fusion_user_embs.npy"), fusion_user)
np.save(os.path.join(DATA,"fusion_item_embs.npy"), fusion_item)
print("✅ v2 融合向量已保存:", fusion_user.shape, fusion_item.shape)


Epoch 1/20 | InfoNCE: 6.7347
Epoch 2/20 | InfoNCE: 8.6787
Epoch 3/20 | InfoNCE: 7.2398
Epoch 4/20 | InfoNCE: 6.7548
Epoch 5/20 | InfoNCE: 6.6943
Epoch 6/20 | InfoNCE: 6.6106
Epoch 7/20 | InfoNCE: 6.2898
Epoch 8/20 | InfoNCE: 6.0925
Epoch 9/20 | InfoNCE: 6.1368
Epoch 10/20 | InfoNCE: 6.2261
Epoch 11/20 | InfoNCE: 6.2332
Epoch 12/20 | InfoNCE: 6.1712
Epoch 13/20 | InfoNCE: 6.0971
Epoch 14/20 | InfoNCE: 6.0473
Epoch 15/20 | InfoNCE: 6.0287
Epoch 16/20 | InfoNCE: 6.0337
Epoch 17/20 | InfoNCE: 6.0499
Epoch 18/20 | InfoNCE: 6.0645
Epoch 19/20 | InfoNCE: 6.0673
Epoch 20/20 | InfoNCE: 6.0541
✅ v2 融合向量已保存: (334, 64) (3415, 64)


In [10]:
# ==========================
# Stage 5 (v2): 上下文敏感的全库 Top-K 评估（统一打分与相似度）
# 使用: FusionHead(user_gcn + ctx_pool) 与 fusion_item_embs 的内积（等同余弦因已归一化）
# ==========================
import os, math, numpy as np, torch, json

DEVICE=torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA="/content/data_v2"
TOPK=[5,10,20]; BATCH_U=512

# 加载
fuser=np.load(os.path.join(DATA,"fusion_user_embs.npy"))     # (U, D) 已归一化
fitem=np.load(os.path.join(DATA,"fusion_item_embs.npy"))     # (I, D) 已归一化
test_pairs=np.load(os.path.join(DATA,"test_pairs.npy"), allow_pickle=True).item()

U,D=fuser.shape; I,_=fitem.shape
user_t=torch.tensor(fuser, dtype=torch.float32, device=DEVICE)
item_t=torch.tensor(fitem, dtype=torch.float32, device=DEVICE)

def recall_ndcg_row(scores, true_idx):
    order=np.argsort(-scores)
    out={}
    for k in TOPK:
        hit=1.0 if true_idx in order[:k] else 0.0
        if hit>0:
            pos=np.where(order==true_idx)[0][0]; ndcg=1.0/math.log2(pos+2)
        else:
            ndcg=0.0
        out[k]=(hit, ndcg)
    return out

# 构造“评估用户列表”
eval_users=sorted(test_pairs.keys())
metrics_sum={k:{"rec":0.0,"ndcg":0.0} for k in TOPK}

# 全库分批
for s in range(0, len(eval_users), BATCH_U):
    batch_u=eval_users[s:s+BATCH_U]
    u_idx=[int(u) for u in batch_u if int(u)<U]
    if not u_idx: continue
    sims=(user_t[u_idx] @ item_t.T).detach().cpu().numpy()  # (B,I)
    for bi, uid in enumerate(u_idx):
        _, tgt = test_pairs[uid]
        res=recall_ndcg_row(sims[bi], int(tgt))
        for k in TOPK:
            metrics_sum[k]["rec"]  += res[k][0]
            metrics_sum[k]["ndcg"] += res[k][1]

N=len(eval_users)
results={f"Recall@{k}": metrics_sum[k]["rec"]/N for k in TOPK}
results.update({f"NDCG@{k}": metrics_sum[k]["ndcg"]/N for k in TOPK})
print("📈 v2 全库 Top-K：")
for k in TOPK:
    print(f"  Recall@{k}: {results[f'Recall@{k}']:.4f} | NDCG@{k}: {results[f'NDCG@{k}']:.4f}")

with open(os.path.join(DATA,"stage5_v2_results.json"),"w") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)
print("💾 指标已保存：stage5_v2_results.json")


📈 v2 全库 Top-K：
  Recall@5: 0.0030 | NDCG@5: 0.0012
  Recall@10: 0.0060 | NDCG@10: 0.0022
  Recall@20: 0.0090 | NDCG@20: 0.0030
💾 指标已保存：stage5_v2_results.json
